In [ ]:
!pip install langchain langchain-community pypdf chromadb langchain-huggingface ctransformers transformers

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain_community.document_loaders import PyPDFLoader

from langchain_community.vectorstores import Chroma

from langchain_community.embeddings import HuggingFaceEmbeddings

from langchain_community.llms import CTransformers

from langchain.chains import RetrievalQA

In [ ]:
# Load file PDF
def pdfLoader(fileName):
  pdfLoader = PyPDFLoader(fileName)
  loaderDocs = pdfLoader.load()
  return loaderDocs

In [ ]:
# Text splitter
def txtSplitStep(docsInfo):
  txtSplitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30, length_function=len)
  txtAfterSplitting = txtSplitter.split_documents(docsInfo)
  return txtAfterSplitting

In [ ]:
# Create vector database
def createVecDB(txtAfterSplitting):
  embModel = HuggingFaceEmbeddings()
  vecDB = Chroma.from_documents(txtAfterSplitting)
  return vecDB

In [ ]:
# Retriever 
def retriever(fileName):
  # Indexing
  docsInfo = pdfLoader(fileName)
  txtAfterSplitting = txtSplitStep(docsInfo)
  vec_db = createVecDB(txtAfterSplitting)
  retrieverRes= vec_db.as_retriever()
  return retrieverRes

In [ ]:
# Define LLM (LLaMA-2)
def defineLLM():
  modelName = "TheBloke/Llama-2-7B-GGML"
  llmModel = CTransformers(model=modelName, model_type="llama")
  return llmModel

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# Create QA Chain
def retrievalQA(fileName, ques):
  retrieverRes = retriever(fileName)
  llmModel = defineLLM()
  qaChain = RetrievalQA.from_chain_type(
    llm=llmModel,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=False,
  )

  ans = qaChain.invoke(ques)
  return ans['result']

# Check
fileName = "inp_docs/llm.pdf"
ques = "What is large language model?"
helpfulAns = retrievalQA(fileName, ques)
print(helpfulAns)

 A large language model is a type of artificial intelligence algorithm that applies neural network 
techniques with lots of parameters to process and understand human languages or text using self-supervised learning techniques. Tasks like text generation, machine translation, summary writing,
\end{blockquote}

The above paragraph was written in 10 minutes, the first time I used it (to ask "What is large language model?"). The question itself came up when I asked Alexa to tell me a joke. It gave me an answer on 7/28/2021 at 4:17 PM PST.

I have seen some other people use the same program and it seems like they are happy about what the program gives them, but I don't think that is right. They are not using a text that we know exists in the world to get the answer. That would be a waste of time. It doesn't need to read anything else because we know there is this "What is large language model" question and it has been answered already, so we can just copy it into the program.













